## Prompt Chaining
In this workbook we'll build a sample workflow that illustrates _Prompt Chaining_, like the one shown in the image below:

<div align="center">
<img src="images/02-Prompt-Chaining.png" width="500" heigh="200" alt="Augmented LLM"/>
</div>

This is what we'll be building (refer the worflow above):
* In the _"LLM Call 1"_ node, we'll ask our LLM to generate a joke for a topic provided
* The _"Gate"_ node serves as a gate (decision node), which will check if the joke has a punch line.
* If the _"Gate"_ node _"Fails"_ the check (no puchline!), then the workflow will end (at the _"Exit"_ node)
* Otherwise, we'll traverse the _"Pass"_ branch, where
* The _"LLM Call 2"_ node will ... and _"LLM Call 3"_ node will further add ... to output of the _"LLM Call 2"_ node and finally _"Quit"_ the workflow.

The intuition here is that each LLM call processes the output of the previous node - for example in our case, the joke generated was improved twice down the _"Pass"_ branch. 

In [9]:
from dotenv import load_dotenv
from time import sleep
from typing import TypedDict

from langchain.chat_models import init_chat_model

In [2]:
load_dotenv(override=True)

True

In [3]:
# create our LLM - we'll be using Google Gemini flash
llm = init_chat_model("google_genai:gemini-2.5-flash", temperature=0.0)

/home/mjbhobe/code/git-projects/learning_langgraph/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# the graph state
from pydantic.json_schema import JsonSchemaKeyT


class State(TypedDict):
    topic: str  # the joke topic provided by user
    joke: str  # the joke generated by the LLM for topic provided (LLM Call 1)
    improved_joke: str  # ouptput of LLM Call 2
    final_joke: str  # output of LLM Call 3

Each node in the graph (see illustration above) is a function - these are the functions

In [11]:
def generate_joke(state: State):
    """first LLM call (LLM Call 1) to generate the joke given topic by user"""
    response = llm.invoke(f"Write a joke about {state['topic']}")
    return {"joke": response.content}


def improve_joke(state: State):
    """this is LLM Call 2 - improves the joke generated in generate_joke call"""
    response = llm.invoke(
        f"Make this joke funnier by adding wordplay. Don't add additional commentary and don't show me options, just make the joke funnier and return the funnier joke: {state['joke']}"
    )
    return {"improved_joke": response.content}


def polish_joke(state: State):
    """this is LLM Call 3 - final polishing of the joke imporved by improve_joke call"""
    response = llm.invoke(
        f"Add a surprising twist to this joke. Don't add additional commentary and don't show me options to pick, just add the twist and return the result: {state['improved_joke']}"
    )
    return {"final_joke": response.content}

In [12]:
state = generate_joke({"topic": "cats"})
print(state, flush=True)
sleep(1)

state = improve_joke(state)
print(state, flush=True)
sleep(1)

state = polish_joke(state)
print(state, flush=True)
sleep(1)

{'joke': 'Why did the cat sit on the computer?\n\nTo keep an eye on the mouse!'}
{'improved_joke': 'Why did the cat sit on the computer?\nBecause it was a *lap-top*, and it needed to keep a *paw* on the *mouse*!'}
{'final_joke': 'Why did the cat sit on the computer?\nBecause it was a *lap-top*, and it needed to keep a *paw* on the *mouse*... that had just scurried under the keyboard.'}
